# "Fun and games"
> "Solving a combinatorial puzzle using a SAT solver"

Quite recently, a friend messaged me with a nice combinatorial puzzle. He was organizing a get-away weekend, and had the following problem: given 6 teams and 5 games which are all 1 versus 1, how does one obtain a schedule in which every team plays every game once, *and* every team plays every other team once?

I promised him I would look into it, and needed to hurry (in a dramatic twist, he needed his answer before nightfall). After an hour or 2 with the computer I was able to solve the problem satisfactorily (as you will find out below, that is a pun). Below I'll walk you through some of the steps and the thinking process.


# Approach
As any good mathematician does, I started by defining clear, unambiguous notation. In particular, I settled on denoting the teams simply by their numbers $i, j = 1, 2, 3, 4, 5, 6$, and games by $G = A, B, C, D, E$. There is also the additional constraint of *rounds*: a team can't be in two places at the same time, so you have to have multiple rounds of games. I simply denote these $n= 1, 2,...$. As we will see, we can get away with exactly 5 rounds.

Sometimes, the problem at hand looks very similar to another problem you have seen before, and for which you can simply look up the solution. I wasn't able to see something, so I decided to go for more general approaches.

After a false start of looking at [integer programming](https://en.wikipedia.org/wiki/Integer_programming) formulations of the problems, I remembered that last year I watched a super nice talk by Raymond Hettinger: [Modern solvers: Problems well-defined are problems solved](https://www.youtube.com/watch?v=_GP9OpZPUYc). In that talk, he talks about leveraging generic solvers to solve all kinds of problems.  
One of the problems he looks at is satisfiability, or the question whether a certain logical formula can be true, and if so, what assumptions make it true. That is a good fit for this problem, so I decided to use the code he made available at [rhettinger.github.io](https://rhettinger.github.io/einstein.html). If you have a SAT-like problem of your own to solve, I would recommend you use it, it makes the whole business quite intuitive.


# Using a SAT solver
To use a SAT solver, you need to express the constraints of your problem in logic. You have some basic facts about the game, like 
> team 3 played game B in round 4

and use them to express things like 
> not ((team 3 played game B in round 4) and (team 3 played game C in round 4))

which amounts to saying that team 3 can't play both game B and game C in round 4. Combining statements like this, you form a big logical formula that exactly expresses the contraints of your problem. The SAT solver then finds which basic facts must be true so that the big logical formula is true.

To put this into practice, let's apply this small example to our SAT solver. We start by writing down our predicates.


In [1]:
from util import *

x = "team 3 played game B in round 4"
y = "team 3 played game C in round 4"

The solver expects a formula in "conjunctive normal form". That sounds pretty scary, but it's not so bad: "conjunction" is another way of saying "and". A conjunction is nothing but a list of logical statements `[A, B, C]` saying that A and B and C are all true. To be a CNF, each of the formulas `A, B, C` cannot contain any further conjunctions (and-operators). That's it!

In our problem, we have a statement that's not in CNF: we would write the statement

> ~(x and y)

But this contains a `~` (not) outside the `and`. The conjunctive normal form of this is `(~x or ~y)`, with all the `and`-clauses at the outermost level (there are none), and only `or`-clauses (disjunctions) in the middle, and negations only on the basic predicates. For the computer, this logical phrase becomes:


In [2]:
cnf = [(neg(x), neg(y))]
cnf

[('~team 3 played game B in round 4', '~team 3 played game C in round 4')]

Where we have used the `neg` function that Raymond already wrote for us. The outermost level is a list of conjunctions, while the inner level is a tuple of disjunctions.  

We can now input these into our SAT solver.


In [3]:
solve_all(cnf, include_neg=True)

[['~team 3 played game B in round 4', '~team 3 played game C in round 4'],
 ['~team 3 played game B in round 4', 'team 3 played game C in round 4'],
 ['team 3 played game B in round 4', '~team 3 played game C in round 4']]

The SAT solver is telling us there are three possible solutions: 
- team 3 doesn't play game B or C in round 4 (I suppose they could be taking a break?)
- team 3 plays game C in round 4
- team 3 plays game B in round 4


# Expressing the problem
We're now ready to address the full problem. For convenience, we define the statement "team i plays game G in round n" as `x(n, i, G)`, writing a little helper function to generate it. We are going to build the final formula incrementally, so we start with an empty list.

In [4]:
from itertools import product, combinations
from util import *

In [5]:
games = ["A", "B", "C", "D", "E"]
teams = [1, 2, 3, 4, 5, 6]
rounds = [1, 2, 3, 4, 5]

x = lambda n, i, G: f"in round {n}, team {i} plays game {G}"

cnf = []

x(4, 2, "B")

'in round 4, team 2 plays game B'

There are several things we need to express about the game. The first two are constraints based on our formulation, and the other two specify our desired solution.
- Each round each team can only play one game
- Games are one versus one
- Every team has played every game
- Each team has played every other team

Let's start by implementing the fact that a team can only play one game per round. For this we use the `one_of` function, which expresses that *exactly* one of the clauses is true.

In [6]:
for n, i in product(rounds, teams):
    cnf += one_of([x(n, i, G) for G in games])
    
len(cnf)

330

Already we have 330 clauses, but of course this is peanuts for our computer. To express the fact that games are one versus one, we say that for every round, and for every game, there are exactly two teams playing any particular game [1]. We can use the `Q` class from `util.py` to express this. `Q` takes a list of predicates, and states how many of them are true.

In [7]:
for n, G in product(rounds, games):
    cnf += (Q([x(n, i, G) for i in teams]) == 2)
    
len(cnf), cnf[:3]

(980,
 [('~in round 1, team 1 plays game A', '~in round 1, team 1 plays game B'),
  ('~in round 1, team 1 plays game A', '~in round 1, team 1 plays game C'),
  ('~in round 1, team 1 plays game A', '~in round 1, team 1 plays game D')])

Now to add one of our desired properties, that every team has played every game. We use the `some_of` function here, which says that *at least* one of the clauses is true.

In [8]:
for i, G in product(teams, games):
    cnf += some_of([x(n, i, G) for n in rounds])

len(cnf)

1010

So far so good, but now it gets tricky! How do we express the fact that every team has played every other team? Saying this must mean there are some round n and some game G for which both "team i plays game G in round n" and "team j plays game G in round n" must be true. We would like to use the `some_of` function here, but it doesn't apply to composites like `x(n, i, G) and x(n, j, G)`, only lists of individual `x`'s.  
Instead, we have to turn to the `from_dnf` function. A DNF is similar to a CNF, but with the roles of the `or` and `and` reversed. We can easily express the fact in DNF: we enter the clauses `x(n, i, G) and x(n, j, G)` for every $n$ and $G$. For 5 rounds and 5 games, that will give us 25 clauses.  
Applying the `from_dnf` function allows us to take this DNF and turn it into an equivalent CNF. However, this really blows up the logical statement: it is exponential in the number of clauses, and for our problem we have 15 times $15 \cdot 2^{15} \approx 503\cdot 10^6$ clauses! As a result, the code below will quickly exceed memory capacity.

# Pivoting
Fear not, because all is not lost! By swapping out the fundamental statements to be "in round n, team i plays team j in game G", it becomes very easy to express the condition that all teams play eachother. We simply need to include both "team i plays team j" and "team j plays team i".


In [9]:
games = ["A", "B", "C", "D", "E"]
teams = [1, 2, 3, 4, 5, 6]
rounds = [1, 2, 3, 4, 5]

x = lambda n, i, j, G: f"In round {n}, team {i} plays team {j} in game {G}"

cnf = []
for i in teams:
    other_teams = [j for j in teams if i != j]
    for j in other_teams:
        cnf += some_of([x(n, i, j, G) for n, G in product(rounds, games)]
                       +[x(n, j, i, G) for n, G in product(rounds, games)])
    
len(cnf)

30

Now to the other rules. To remind ourselves, they are:

* Games are one versus one
* Each team can only play one game per round
* Every team has played every game

There is a fourth one introduced by our notation: the statement `x(n, i, i, G)` would imply that team $i$ played itself, and this can of course not happen.

* A team cannot play against itself.

Now the first one is easy to implement: it is encoded in the structure of the statement itself! The others follow.


In [10]:
# Every team plays every game:
# There exist n, j such that in round n, i plays j in game G
for i, G in product(teams, games):
    cnf += some_of([x(n, i, j, G) for n, j in product(rounds, teams)]
                   + [x(n, j, i, G) for n, j in product(rounds, teams)])
    

# Each team can only play one game per round
# Exactly one of x(n, i, *, *) or x(n, *, i, *) is true per round
for i, n in product(teams, rounds):
    other_teams = [j for j in teams if j != i]
    cnf += one_of([x(n, i, j, G) for j, G in product(other_teams, games)] +
                 [x(n, j, i, G) for j, G in product(other_teams, games)])


# A team cannot play against itself
for i, n, G in product(teams, rounds, games):
    cnf += [[neg(x(n, i, i, G))]]
    
    
len(cnf)

36990

That seems rather manageable. And indeed, our SAT solver finds one quickly. We simply apply `solve_one` to get a single solution, sorting it for readability. By default, the SAT solver only lists true clauses.

In [11]:
sorted(solve_one(cnf))

['In round 1, team 2 plays team 5 in game C',
 'In round 1, team 3 plays team 6 in game E',
 'In round 1, team 4 plays team 1 in game B',
 'In round 2, team 1 plays team 6 in game A',
 'In round 2, team 2 plays team 4 in game E',
 'In round 2, team 3 plays team 5 in game B',
 'In round 3, team 3 plays team 1 in game C',
 'In round 3, team 5 plays team 4 in game A',
 'In round 3, team 6 plays team 2 in game B',
 'In round 4, team 2 plays team 1 in game D',
 'In round 4, team 4 plays team 3 in game D',
 'In round 4, team 5 plays team 6 in game D',
 'In round 5, team 2 plays team 3 in game A',
 'In round 5, team 5 plays team 1 in game E',
 'In round 5, team 6 plays team 4 in game C']

You can check for yourself that it satisfies all the constraints. Note that this is only one of the possible solutions. Just from this one, we can generate another $6! \cdot 5! \cdot 5! \approx 10\cdot 10^6$ more, by reordering rounds, teams, and games.

# Summary and conclusion
We discussed a problem involving 6 teams and 5 teams. The question was if we could find a roster in which every team plays every game, and every team plays every other team. Inspired by [this talk]() from Raymond Hettinger, we applied a SAT solver to the problem. We found out that the encoding of basic facts can matter a lot for the feasilibity of computing a solution, but in the end we developed a satisfactory (get it?) solution in just 34 lines of code.  

If you ever come across a similar problem, I hope this post is helpful in developing a solution. Until next time!

## Notes
1. This also inadvertently introduces the constraint that every game must be played in every round. If the SAT solver pointed out that a feasible solution could not be found, we would have to change it.